In [ ]:
import sys
import plotly.express as px
import time
import numpy as np
sys.path.append('../Functions/py_functions/') # This path is so that within each function file, you can import the other function files with relative paths
sys.path.append('../') # This path is so that we can import the functions folder from the root directory compared to where this file is
from Functions.py_functions.constants import *
from Functions.py_functions.car_configuration import Car
import plotly.graph_objects as go

from Functions.py_functions.previous_cars import sr_9
from Functions.py_functions.tire_model.tire_model_utils import * # this has the tire models
from Functions.py_functions.las_solvers.las import LAS
from Functions.py_functions.las_solvers.octahedral import Octahedral_LAS
from Functions.py_functions.las_solvers.multi_layer import Multi_Layer_LAS
from Functions.py_functions.steady_state_solver.ls_optimize import LS_Solver

In [ ]:
# import SR9 for instance
# car = Car()
car = sr_9()
car.set_tire(H_R20_18X6_7)
# car.debug = True
v_average = 15
mu = 0.65 # mu correction factor, a value of 1 means the road is sandpaper and the actual value should be something lower but im kinda just setting this value to overfit atm
target = 0.001
solver = LS_Solver()
las = Multi_Layer_LAS(solver=solver)

In [ ]:
las.generate_las(car, vel_bins=8, mu=mu, add_layers=3)

In [ ]:
# plot the corners of each LAS through the longitudinal acceleration
# we will generate a line for each corner and for each velocity bin
# we only care about the two ordinate corners because the other two are just mirrored
ay_corners = []
yaw_corners = []
corner_vels = []
for v, vel in enumerate(las.vels):
    ay_corners.append(las.aymax_l[v])
    yaw_corners.append(las.yawmax_l[v])
    corner_vels.append(vel)

# now we plot them, we shall plot both in their AY axis and their YAW axis
# long, lat, yaw, delta, beta
ay_corner = np.array(ay_corners)
print(ay_corner[0, :, 0])
yaw_corner = np.array(yaw_corners)
fig1 = go.Figure()
fig2 = go.Figure()
fig3 = go.Figure()
fig4 = go.Figure()
fig5 = go.Figure()
fig6 = go.Figure()
for v, vel in enumerate(corner_vels):
    fig1.add_trace(go.Scatter(x=ay_corner[v, :, 0], y=ay_corner[v, :, 1], mode='lines', name=f'V = {vel:.2f} m/s'))
    fig2.add_trace(go.Scatter(x=ay_corner[v, :, 0], y=ay_corner[v, :, 2], mode='lines', name=f'V = {vel:.2f} m/s'))
    fig3.add_trace(go.Scatter(x=ay_corner[v, :, 0], y=yaw_corner[v, :, 1], mode='lines', name=f'V = {vel:.2f} m/s'))
    fig4.add_trace(go.Scatter(x=ay_corner[v, :, 0], y=yaw_corner[v, :, 2], mode='lines', name=f'V = {vel:.2f} m/s'))
    fig5.add_trace(go.Scatter(x=ay_corner[v, :, 3], y=ay_corner[v, :, 4], mode='lines+markers', name=f'V = {vel:.2f} m/s'))
    fig6.add_trace(go.Scatter(x=yaw_corner[v, :, 3], y=yaw_corner[v, :, 4], mode='lines+markers', name=f'V = {vel:.2f} m/s'))
fig1.update_layout(title='LAS Corner Convergence Ay', xaxis_title='Ax Acceleration (m/s^2)', yaxis_title='Ay (m/s^2)')
fig2.update_layout(title='LAS Corner Convergence Ay', xaxis_title='Ax Acceleration (m/s^2)', yaxis_title='Yaw (rad/s^2)')
fig3.update_layout(title='LAS Corner Convergence Yaw', xaxis_title='Ax Acceleration (m/s^2)', yaxis_title='Ay (m/s^2)')
fig4.update_layout(title='LAS Corner Convergence Yaw', xaxis_title='Ax Acceleration (m/s^2)', yaxis_title='Yaw (rad/s^2)')
fig5.update_layout(title='LAS Corner Convergence Ay', xaxis_title='Beta Angle', yaxis_title='Delta Angle')
fig6.update_layout(title='LAS Corner Convergence Yaw', xaxis_title='Beta Angle', yaxis_title='Delta Angle')
fig1.show()
fig2.show()
fig3.show()
fig4.show()
fig5.show()
fig6.show()